# Names 
Spencer Bernardo-Cheng
Max Goldstein
Robbie Keehan

# Data Preparation

In [56]:
import numpy as np
import pandas as pd
datafb = pd.read_csv("./FB.csv")
datagoogle = pd.read_csv("./GOOGL.csv")
datamsft = pd.read_csv("./MSFT.csv")
dataam = pd.read_csv("./AMZN (1).csv")
datanflx = pd.read_csv("./NFLX.csv")

del datafb['Date']
del datagoogle['Date']
del datamsft['Date']
del datanflx['Date']


data = pd.concat([datafb, datagoogle, datamsft, dataam, datanflx], axis=1, sort=False)
data = data[90:]
del data['Change']
del data['Gain']
del data['Loss']
del data['Average Gain']
del data['Average Loss']
del data['RS']
del data['DM+1']
del data['DM-1']
del data['TR-14']
del data['DM+1-14']
del data['DM-1+14']
del data['dl+1-14']
del data['dl-1-14']
del data['dl14diff']
del data['dl14sum']
del data['DX']
del data['TR']

up = .03
down =-.03
ydata = data['% Change']
ydata = ydata.apply(lambda x:2 if x>=up else (1 if (x< up and x > down) else 0))
ydata = pd.DataFrame(ydata)
ydata['newy'] = ""
# created target data in format to match output of multiple timesteps of 14 days ahead
# [0,1,2,3,4,5,6,7]
# [1,2,3,4,5,6,7,8]
for index, row in ydata.iterrows():
    temp = np.array(ydata['% Change'][index-90:index-90+14].values)
    ydata['newy'].at[index] = temp
    
# removed last 14 because their targets are not able to created
ydata = ydata[0:-13]
data = data[0:-13]
del ydata['% Change']
print(ydata.shape)
print(ydata)


(1789, 1)
                                            newy
90    [0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2]
91    [0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0]
92    [0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0]
93    [0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2]
94    [2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2]
...                                          ...
1874  [2, 0, 2, 0, 2, 1, 0, 0, 2, 2, 2, 2, 0, 0]
1875  [0, 2, 0, 2, 1, 0, 0, 2, 2, 2, 2, 0, 0, 0]
1876  [2, 0, 2, 1, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0]
1877  [0, 2, 1, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0]
1878  [2, 1, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2]

[1789 rows x 1 columns]


# One Hot Encoding The Target Data

In [57]:
from keras.utils.np_utils import to_categorical
print(ydata.head())
ydata['newy'] = ydata['newy'].apply(lambda x: to_categorical(x,3))
print(ydata)

                                          newy
90  [0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2]
91  [0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0]
92  [0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0]
93  [0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2]
94  [2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2]
                                                   newy
90    [[1.0, 0.0, 0.0], [1.0, 0.0, 0.0], [1.0, 0.0, ...
91    [[1.0, 0.0, 0.0], [1.0, 0.0, 0.0], [1.0, 0.0, ...
92    [[1.0, 0.0, 0.0], [1.0, 0.0, 0.0], [0.0, 0.0, ...
93    [[1.0, 0.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, ...
94    [[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 0.0, ...
...                                                 ...
1874  [[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 0.0, ...
1875  [[1.0, 0.0, 0.0], [0.0, 0.0, 1.0], [1.0, 0.0, ...
1876  [[0.0, 0.0, 1.0], [1.0, 0.0, 0.0], [0.0, 0.0, ...
1877  [[1.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, ...
1878  [[0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [1.0, 0.0, ...

[1789 rows x 1 columns]


# Min-Max Scaling the Time Series

In [59]:
from keras.models import Sequential
from keras.layers import Dense
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.preprocessing.sequence import TimeseriesGenerator

n_input = 2
series = np.array(data)
target = np.array(ydata)
generator = TimeseriesGenerator(series, target, length=90, batch_size=1)
xtime = []
ytarget =[]
for i in range(len(generator)):
    x, y = generator[i]
    xtime.append(x[0])
    ytarget.append(y[0])
xtime = np.array(xtime)
ytarget = np.array(ytarget)

print(ytarget)

[[array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)]
 [array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)]
 [array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)]
 ...
 [array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 

In [39]:

model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(90, 37)))
model.add(Dense(2))
model.summary()
model.compile(optimizer='adam', loss='mse')

# # fit model
# model.fit(generator, steps_per_epoch=1, epochs=500, verbose=0)
# # make a one step prediction out of sample
# x_input = array([[90, 95], [100, 105]]).reshape((1, n_input, n_features))
# yhat = model.predict(x_input, verbose=0)
# print(yhat)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 100)               55200     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 202       
Total params: 55,402
Trainable params: 55,402
Non-trainable params: 0
_________________________________________________________________
